In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-a9c1  GCE       2                                       RUNNING  us-central1-a


In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import gc

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Nov 21 17:33 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'irproject2066' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("id", "text").rdd

N_temp = parquetFile.count()

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp_final.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp_final.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

sc.addFile("/home/dataproc/write_to_memory.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp_final import InvertedIndex

from write_to_memory import *

# First Paragraph Index

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  counter = Counter(tokens)

  returnList = []
  for c in counter:
    if c not in all_stopwords:
      returnList.append((c, (id,counter[c])))

  return returnList

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[0])

def calculate_df(postings):
  return postings.map(lambda x: (x[0],len(x[1])))

  ####### we can add filtering on rare terms

In [ ]:
def partition_postings_and_write(postings, directory):
  with_bucket_id = postings.map(lambda w: (token2bucket_id(w[0]) , (w[0] , w[1]))).groupByKey()
  buckets_mapped = with_bucket_id.map(lambda w: InvertedIndex.write_a_posting_list(w, bucket_name, directory))
  return buckets_mapped

In [ ]:
def take_first_paragraph(text):
  splits = text.split("\n\n")
  combined_text = ""
  for mini_text in splits:
    splited_list = mini_text.split(" ")
    if len(splited_list) < 8:
      break
    else:
      combined_text = combined_text + " "+ mini_text
  return combined_text


In [ ]:
# word counts map
paragraph = doc_text_pairs.map(lambda x: (x[0], take_first_paragraph(x[1])))
print("done1")
del doc_text_pairs
gc.collect()
word_counts = paragraph.flatMap(lambda x: word_count(x[0], x[1]))
print("done2")
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
print("done3")
w2df = calculate_df(postings)
print("done4")
w2df_dict = w2df.collectAsMap()
print("done5")
_ = partition_postings_and_write(postings,'postings_paragraph2').collect()
print("done6")

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_paragraph2'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict
inverted.write_index('.', 'paragraph_index')

index_src = "paragraph_index.pkl"
index_dst = f'gs://{bucket_name}/postings_paragraph2/{index_src}'
!gsutil cp $index_src $index_dst

del super_posting_locs
del _
del word_counts
del postings
del w2df
gc.collect()